In [57]:
import dimod
import numpy as np
import dwave_token
import dwave.inspector

from dwave.system import DWaveSampler, EmbeddingComposite
# from dwave.system import LeapHybridCQMSampler

sampler = EmbeddingComposite(DWaveSampler(token=dwave_token.value))

In [65]:
n = 5
scaling = 10

variables = ['x_' + str(i) for i in range(n)]
Q = scaling*(-1+2*np.random.rand(n,n))
bqm = {(variables[i],variables[j]): 0 for i in range(n) for j in range(n)}
for i in range(n):
    for j in range(n):
        bqm[(variables[i],variables[j])] = Q[i,j]
print(Q)

[[ 1.77522819 -4.32948214]
 [ 0.85416873 -4.22825839]]


In [66]:
sampleset = sampler.sample_qubo(bqm, num_reads=100)
print(sampleset.lowest().first.energy)

BinaryQuadraticModelStructureError: Problem graph incompatible with solver. Please use 'EmbeddingComposite' to map the problem graph to the solver.

In [50]:
print(sampleset)
print(sampleset.info)

  x_0 x_1 x_2 x_3 x_4    energy num_oc. chain_.
0   1   1   1   0   0 -6.187647      45     0.0
1   0   1   0   0   0 -6.083592      49     0.0
2   1   0   1   0   0 -5.275195       5     0.0
3   0   0   0   0   1 -3.427488       1     0.0
['BINARY', 4 rows, 100 samples, 5 variables]


In [54]:
dwave.inspector.show(sampleset)

RuntimeError: threads can only be started once